In [8]:
import pandas as pd
from pathlib import Path
import ast
import os
from datetime import date

In [ ]:
news_df = pd.read_csv('/home/rohit/Dropbox/results/topics/news_2016_predictions.csv')
news_df['top1_topic'] = news_df.apply(lambda x: "_".join(x.top1_topic.replace(',', '').split(" ")), axis=1)
news_df['Date'] = news_df.apply(lambda x: date(x['year'], x['month'], x['day']).strftime('%B %d, %Y'), axis=1)
news_df = news_df.drop(['Unnamed: 0', 'day', 'month', 'year', 'parliament', 'top1_acc', 'top2_topic', 'top2_acc', 'top3_topic', 'top3_acc'], axis=1)
news_df.columns = ['source_id', 'Source', 'Program Name', 'Transcript', 'topic', 'Date']

In [ ]:
news_df

In [ ]:
id_to_name_map = {
    400553: 'Belfast Telegraph',
    377101: 'Scotsman',
    418973: 'Record',
    244365: 'Wales',
    8200: 'Independent',
    412338: 'Wales',
    138794: 'Mail',
    232241: 'Express',
    334988: 'Telegraph',
    331369: 'Sun',
    138620: 'Guardian',
    419001: 'Mirror',
    8010: 'Guardian',
    142728: 'Herald',
    408506: 'Express',
    143296: 'Observer',
    363952: 'Star',
    145251: 'People',
    232240: 'Express',
    145253: 'Record',
    389195: 'Telegraph',
    145254: 'Mirror',
    344305: 'Scotland',
    8109: 'Telegraph',
    397135: 'Mail',
    163795: 'Belfast Telegraph',
    412334: 'Post',
    408508: 'Star',
    411938: 'Standard'
}

news_ids = list(id_to_name_map.keys())

topics_index_to_name_map = {
    0: 'Agriculture_animals_food_and_rural_affairs',
    1: 'Asylum_immigration_and_nationality',
    2: 'Business_industry_and_consumers',
    3: 'Communities_and_families',
    4: 'Crime_civil_law_justice_and_rights',
    5: 'Culture_media_and_sport',
    6: 'Defence',
    7: 'Economy_and_finance',
    8: 'Education',
    9: 'Employment_and_training',
    10: 'Energy_and_environment',
    11: 'European_Union',
    12: 'Health_services_and_medicine',
    13: 'Housing_and_planning',
    14: 'International_affairs',
    15: 'Parliament_government_and_politics',
    16: 'Science_and_technology',
    17: 'Social_security_and_pensions',
    18: 'Social_services',
    19: 'Transport',
    20: 'Others'
}

topics = list(topics_index_to_name_map.values())

In [ ]:
counts = pd.crosstab(news_df.Source, news_df.topic)
counts

In [ ]:
counts.to_csv('news_2016_topics_counts.csv')

In [ ]:
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
month_name_to_index = {months[i]:i+1 for i in range(12)}
for topic in topics:
    for news_id in news_ids:
        print(topic, news_id)
        path = './news_topics/{}_{}/2016/transcripts'.format(news_id, topic)
        Path(path).mkdir(parents=True, exist_ok=True)
        news_for_topic_and_id = news_df.loc[(news_df.topic == topic) & (news_df.source_id == news_id)]
        for month in months:
            news_for_topic_and_id_and_month = news_for_topic_and_id[news_for_topic_and_id.Date.str.contains(month)]
            if (len(news_for_topic_and_id_and_month) == 0):
                continue
            news_for_topic_and_id_and_month.to_csv(os.path.join(path, '{}-2016 {}_{}.csv'.format(month, news_id, topic)))

In [114]:
rows = []

for topic in topics:
    for news_id in news_ids:
        row = [id_to_name_map[news_id]+ ' ' +topic, str(news_id)+'_'+topic, 'Nexis', '', '']
        rows.append(row)
        
df = pd.DataFrame(rows, columns=['Source Name', 'SourceId', 'Database', 'Start', 'End'])
df

,Source Name,SourceId,Database,Start,End
0,Belfast Telegraph Agriculture_animals_food_and...,400553_Agriculture_animals_food_and_rural_affairs,Nexis,,
1,Scotsman Agriculture_animals_food_and_rural_af...,377101_Agriculture_animals_food_and_rural_affairs,Nexis,,
2,Record Agriculture_animals_food_and_rural_affairs,418973_Agriculture_animals_food_and_rural_affairs,Nexis,,
3,Wales Agriculture_animals_food_and_rural_affairs,244365_Agriculture_animals_food_and_rural_affairs,Nexis,,
4,Independent Agriculture_animals_food_and_rural...,8200_Agriculture_animals_food_and_rural_affairs,Nexis,,
...,...,...,...,...,...
604,Mail Others,397135_Others,Nexis,,
605,Belfast Telegraph Others,163795_Others,Nexis,,
606,Post Others,412334_Others,Nexis,,
607,Star Others,408508_Others,Nexis,,


In [115]:
df.to_csv('./sources2.csv')